In [ ]:
import os
import numpy as np
import pandas as pd
import sys
base_path_project = "/home/till/projects/night-drive/"    # path to project fodler
sys.path.append(base_path_project)
import utils.eval_utils
from utils.eval_dataset import EvalDataset

In [ ]:
root_dir = "/home/till/SharedFolder/CurrentDatasets/"

In [9]:
# load original BDD data
orig = EvalDataset(root_dir, database='bdd_all')

>> Loading bdd_all label data set
/images/100k/val/


In [10]:
orig.data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
# for each data split
list_splits = [*orig.list_splits, *orig.list_splits_over]

In [ ]:
# write prints to log file in analysis folder
old_stdout = sys.stdout
log_file = open("datasets_integrity_test_log.log", "w")
sys.stdout = log_file
for split in list_splits:
    print("\n\n=============================================")
    print("TESTING INTEGRITY OF SPLIT {}".format(split))
    # load data
    cur = EvalDataset(root_dir, database=split)
    # # check whether the same image occurs with different weather calsses in the json to be investigated
    # print("\nPerforming check on class agreement within data set")
    # for i in range(cur.data.shape[0]):
    #     for j in range(cur.data.shape[0]):
    #         if i != j:
    #             if cur.data.loc[i,"name"] == cur.data.loc[j,"name"] and cur.data.loc[i,"weather"] != cur.data.loc[j,"weather"]:
    #                 print("weather mismatch error")
    #                 print(cur.data.loc[i, "name"], cur.data.loc[i, "weather"])
    #                 print(cur.data.loc[j, "name"], cur.data.loc[j, "weather"])
    #                 
    # check whether the weather and timeofday classes in the json under investigation match those in the original BDD jsons
    print("\nPerforming check on class agreement with original data set")
    for j in range(cur.data.shape[0]):
        im_name = cur.data.loc[j,"name"]
        im_weather = cur.data.loc[j,"weather"]
        inx = np.array(orig.data.query("name == @im_name").index)
        orig_weather = orig.data.loc[inx[0], "weather"]
        orig_timeofday = orig.data.loc[inx[0], "timeofday"]
        orig_name = orig.data.loc[inx[0], "name"]
        if im_weather != orig_weather and \
            (set([im_weather]).isdisjoint(["cloudy", "overcast", "partly cloudy"]) and set([orig_weather]).isdisjoint(["cloudy", "overcast", "partly cloudy"])):
            print("weather mismatch error")
            print(im_name, im_weather)
            print(orig_name, orig_weather)
        if orig_timeofday != orig_timeofday:
            print("timeofday mismatch error")
            print(im_name, im_weather)
            print(orig_name, orig_weather)
            
    # check whether over samples are uniformly distributed
    if "over" in split:
        count, division = np.histogram(cur.data.name.value_counts())
        print("\nValue counts of images in oversampled data set")
        print("divisions:", division.tolist())
        print("counts:", count.tolist())
        print("\nFive most frequent images in oversampled data set")
        cur.data.name.value_counts()[:5]
    # Check class distribution of timeofday x weather classes
    print("\nCrosstab counts of images in oversampled data set")
    print(cur.get_crosstab_timeofdayxweather())

# close log file
sys.stdout = old_stdout
log_file.close()